In [47]:
import sys
import os

sys.path.append("/home/jupyter/news/src")
from newsapiscraper import NewsApiScraper
from newscollector import NewsCollector
from googletranslation import GoogleTranslate
from clustering import Clustering
from googleembeddings import GoogleEmbeddings
from strike_rag import StrikeRAG
from fire_rag import FireRAG
from filtering import Filter
from matching import Matching
from configuration import Configuration
import sectors_to_discard
from utils import split_liste, groupByName, generate_description, geoloc
import strike_relevancy


import pandas as pd
import numpy as np
import json
import requests
from copy import deepcopy
from geopy.geocoders import Nominatim
from dotenv import load_dotenv

In [48]:
load_dotenv()
NEWS_API_KEY = os.getenv('NEWS_API_KEY')
os.environ['http_proxy'] = os.getenv('http_proxy')
os.environ['https_proxy'] = os.getenv('https_proxy')
os.environ['HTTP_PROXY'] = os.getenv('HTTP_PROXY')
os.environ['HTTPS_PROXY'] = os.getenv('HTTPS_PROXY')

# Creating config file for the first time

## Generation initial config dict
We generate a dict that will be use to generate the finale config file. This final config should be generate once and be used at every moment if we do not add some new keys.

In [4]:
dict_config = {}
dict_config['keywords'] = ['strike', 'picket line', 'employee protest']
dict_config['country_lang'] = [
    {'country': 'BE','lang': ['fr']}, # Belgium
    {'country': 'CH','lang': ['fr', 'de', 'it']}, # Searching news with differentes languge in the same country where threre are several official language
    {'country': 'BG','lang': ['bg']},  # Bulgaria
    {'country': 'BR','lang': ['pt']},
    {'country' : 'CL',  'lang' : ['es']},
    {'country' : 'CN',  'lang' : ['zh']},
    {'country' : 'CO',  'lang' : ['es']},
    {'country' : 'CZ',  'lang' : ['cs']},
    {'country' : 'DE',  'lang' : ['de']},
    {'country' : 'DZ',  'lang' : ['ar']},
    {'country' : 'EE',  'lang' : ['et']},
    {'country' : 'ES',  'lang' : ['es']},
    {'country' : 'FR',  'lang' : ['fr']},
    {'country' : 'GB',  'lang' : ['en']},
    {'country' : 'HU',  'lang' : ['hu']},
    {'country' : 'ID',  'lang' : ['id']},
    {'country' : 'IN',  'lang' : ['hi']},
    {'country' : 'IT',  'lang' : ['it']},
    {'country' : 'JP',  'lang' : ['ja']},
    {'country' : 'KR',  'lang' : ['ko']},
    {'country' : 'LT',  'lang' : ['lt']}, 
    {'country' : 'MA',  'lang' : ['ar']},
    {'country' : 'MC',  'lang' : ['fr']}, 
    {'country' : 'MX',  'lang' : ['es']},
    {'country' : 'MY',  'lang' : ['ms']},
    {'country' : 'NL',  'lang' : ['nl']},
    {'country' : 'PL',  'lang' : ['pl']},
    {'country' : 'PT',  'lang' : ['pt']},
    {'country' : 'RO',  'lang' : ['ro']},
    {'country' : 'SI',  'lang' : ['sl']},
    {'country' : 'SK',  'lang' : ['sk']},
    {'country' : 'TH',  'lang' : ['th']},
    {'country' : 'TN',  'lang' : ['ar']},
    {'country' : 'TR',  'lang' : ['tr']},
    {'country' : 'UA',  'lang' : ['uk']},
    {'country' : 'VN',  'lang' : ['vi']},
    {'country' : 'SE',  'lang' : ['sv']},
    {'country' : 'SV',  'lang' : ['es']},
    {'country' : 'CA',  'lang' : ['fr']},
    {'country' : 'LU',  'lang' : ['fr']}
]

dict_config['NEWS_API_KEY'] = NEWS_API_KEY # if you use News API services

dict_config['rag_cong'] = {'vertexai_llm': 'gemini-1.5-flash',
                            'vertexai_embedding_name': 'text-embedding-004',
                            'chunk_size': 2000, 'chunk_overlap': 10, 'max_doc': 5, 'retry': 1}
# dict_config["sectors_to_discard"] = sectors_to_discard.SECTORS_TO_DISCARD # Or spécify a liste of sectors you want to discard
dict_config["decision_function_args"] = {
    'sectors_to_discard': sectors_to_discard.SECTORS_TO_DISCARD, # Or spécify a liste of sectors you want to discard
    'desirable_temporalities' : ['upcoming', 'ongoing', 'unknown']
}

dict_config['project_id']="irn-67050-lab-65"
                          

In [5]:
dict_config

{'keywords': ['strike', 'picket line', 'employee protest'],
 'country_lang': [{'country': 'BE', 'lang': ['fr']},
  {'country': 'CH', 'lang': ['fr', 'de', 'it']},
  {'country': 'BG', 'lang': ['bg']},
  {'country': 'BR', 'lang': ['pt']},
  {'country': 'CL', 'lang': ['es']},
  {'country': 'CN', 'lang': ['zh']},
  {'country': 'CO', 'lang': ['es']},
  {'country': 'CZ', 'lang': ['cs']},
  {'country': 'DE', 'lang': ['de']},
  {'country': 'DZ', 'lang': ['ar']},
  {'country': 'EE', 'lang': ['et']},
  {'country': 'ES', 'lang': ['es']},
  {'country': 'FR', 'lang': ['fr']},
  {'country': 'GB', 'lang': ['en']},
  {'country': 'HU', 'lang': ['hu']},
  {'country': 'ID', 'lang': ['id']},
  {'country': 'IN', 'lang': ['hi']},
  {'country': 'IT', 'lang': ['it']},
  {'country': 'JP', 'lang': ['ja']},
  {'country': 'KR', 'lang': ['ko']},
  {'country': 'LT', 'lang': ['lt']},
  {'country': 'MA', 'lang': ['ar']},
  {'country': 'MC', 'lang': ['fr']},
  {'country': 'MX', 'lang': ['es']},
  {'country': 'MY', 'lan

In [6]:
initial_config_file ="/home/jupyter/news/config/initial_config_file.json"
with open(initial_config_file, 'w') as file :
    json.dump(dict_config, file, indent = 4)

## Generating final config file from initial config file
It it this final config file which will be used to get news

In [7]:
initial_config_file = "/home/jupyter/news/config/initial_config_file.json"
final_config_file = "/home/jupyter/news/config/final_config_file.json"
config = Configuration( initial_config_file = initial_config_file, final_config_file = final_config_file)

# Loading config file

In [8]:
config_file = '/home/jupyter/news/config/final_config_file.json'
with open(config_file, 'r') as file :
    conf = json.load(file)
conf['decision_function'] = strike_relevancy.strike_relevancy
conf

{'keywords': ['strike', 'picket line', 'employee protest'],
 'country_lang': [{'country': 'BE',
   'lang': 'fr',
   'queries': ['grève', 'piquet de grève', 'protestation des employés']},
  {'country': 'CH',
   'lang': 'fr',
   'queries': ['grève', 'piquet de grève', 'protestation des employés']},
  {'country': 'CH',
   'lang': 'de',
   'queries': ['schlagen', 'Streikposten', 'Mitarbeiterprotest']},
  {'country': 'CH',
   'lang': 'it',
   'queries': ['sciopero',
    'cordone di scioperanti',
    'protesta dei dipendenti']},
  {'country': 'BG',
   'lang': 'bg',
   'queries': ['стачка', 'пикет линия', 'протест на служител']},
  {'country': 'BR',
   'lang': 'pt',
   'queries': ['batida', 'linha de piquete', 'protesto de funcionários']},
  {'country': 'CL',
   'lang': 'es',
   'queries': ['huelga', 'piquete', 'protesta de empleados']},
  {'country': 'CN',
   'lang': 'zh',
   'queries': ['strike', 'picket line', 'employee protest']},
  {'country': 'CO',
   'lang': 'es',
   'queries': ['huelg

# Instanciating NewsApiScraper

In [9]:
scrapper = NewsApiScraper(api_key= conf['NEWS_API_KEY'], start_date='2024-08-22',end_date='2024-08-23')

# Collecting news via Newscolllector instance

In [10]:
collector = NewsCollector(config=conf['country_lang'], scraper=scrapper,path_to_save='/home/jupyter/news/data/strike_newsapi_vendre_23_aout.csv')
collector.collect_news()

  0%|          | 0/42 [00:00<?, ?it/s]

{'country': 'BE', 'lang': 'fr', 'queries': ['grève', 'piquet de grève', 'protestation des employés']}
search ended !



100%|██████████| 15/15 [00:07<00:00,  2.10it/s]


News collection ended ! 
BE fr grève
 data_.shape :(15, 6)
search ended !



0it [00:00, ?it/s]


...

News collection ended ! 
LU fr piquet de grève


search ended !



0it [00:00, ?it/s]
100%|██████████| 42/42 [04:03<00:00,  5.80s/it]


News collection ended ! 
LU fr protestation des employés


,dates,titles,links,texts,lang,cat
0,2024-08-23T07:33:30Z,Le Seigneur des Anneaux : La Guerre du Rohirim...,https://www.journaldugeek.com/2024/08/23/le-se...,La Terre du Milieu sera à l’honneur en cette f...,fr,grève
1,2024-08-23T08:00:10Z,Le viol et le meurtre d’une médecin illustrent...,https://www.lemonde.fr/international/article/2...,Manifestation de médecins et d’ambulanciers ap...,fr,grève
2,2024-08-22T14:07:30Z,"En Inde, après le viol et le meurtre d’une méd...",https://www.lemonde.fr/international/article/2...,Des avocats de la Haute Cour de Calcutta lors ...,fr,grève
3,2024-08-22T16:31:40Z,L’été où l’Espagne a dit non au surtourisme,https://www.courrierinternational.com/article/...,Certains étaient déjà descendus dans la rue po...,fr,grève
4,2024-08-22T14:30:10Z,"En Inde, les médecins du principal hôpital de ...",https://www.francetvinfo.fr/monde/inde/en-inde...,Les médecins ont souligné qu'ils mettaient fin...,fr,grève
...,...,...,...,...,...,...
16,2024-08-23T07:11:51Z,FN: 90 procent av Gazaborna har tvingats fly,https://www.aftonbladet.se/nyheter/a/Rr77qd/af...,Palestinier tvingades återigen fly vid en evak...,sv,strejk
17,2024-08-23T04:47:52Z,Brand i flera bilar,https://www.aftonbladet.se/nyheter/a/Rr77qd/af...,TRE NYHETER DU INTE FÅR MISSA: MP-topparnas be...,sv,strejk
18,2024-08-23T04:42:31Z,Brand i härbärge,https://www.aftonbladet.se/nyheter/a/Rr77qd/af...,TRE NYHETER DU INTE FÅR MISSA: MP-topparnas be...,sv,strejk
19,2024-08-23T05:20:05Z,Misstänkt föremål i Kalmar var skarpt,https://www.aftonbladet.se/nyheter/a/Rr77qd/af...,TRE NYHETER DU INTE FÅR MISSA: MP-topparnas be...,sv,strejk


In [11]:
collector.data.shape

(327, 6)

In [12]:
collector.data.columns

Index(['dates', 'titles', 'links', 'texts', 'lang', 'cat'], dtype='object')

In [13]:
collector.data.head()

,dates,titles,links,texts,lang,cat
0,2024-08-23T07:33:30Z,Le Seigneur des Anneaux : La Guerre du Rohirim...,https://www.journaldugeek.com/2024/08/23/le-se...,La Terre du Milieu sera à l’honneur en cette f...,fr,grève
1,2024-08-23T08:00:10Z,Le viol et le meurtre d’une médecin illustrent...,https://www.lemonde.fr/international/article/2...,Manifestation de médecins et d’ambulanciers ap...,fr,grève
2,2024-08-22T14:07:30Z,"En Inde, après le viol et le meurtre d’une méd...",https://www.lemonde.fr/international/article/2...,Des avocats de la Haute Cour de Calcutta lors ...,fr,grève
3,2024-08-22T16:31:40Z,L’été où l’Espagne a dit non au surtourisme,https://www.courrierinternational.com/article/...,Certains étaient déjà descendus dans la rue po...,fr,grève
4,2024-08-22T14:30:10Z,"En Inde, les médecins du principal hôpital de ...",https://www.francetvinfo.fr/monde/inde/en-inde...,Les médecins ont souligné qu'ils mettaient fin...,fr,grève


# Translation 

In [14]:
translation = GoogleTranslate(project_id=conf['project_id'])

In [15]:
conf['project_id']

'irn-67050-lab-65'

In [16]:
df = collector.data
df.shape

(327, 6)

In [17]:
del os.environ['http_proxy']
del os.environ['https_proxy']
del os.environ['HTTP_PROXY']
del os.environ['HTTPS_PROXY']

In [18]:
## Instanciating Translation object in order to translate the collected news

In [19]:
 trans_df = translation.translation(df, limit=30720)

In [20]:
# translation.fails_index

In [21]:
trans_df.shape

(327, 8)

In [22]:
trans_df.columns

Index(['dates', 'titles', 'links', 'texts', 'lang', 'cat', 'translated_title',
       'translated_text'],
      dtype='object')

# Cleaning the translated dataframe
On loses more data when the cleaning happens before translation. In fact, some languages like japanes or chinese do not follow some common rule like have space between words. So one loses those data if cleaning happens base one blank space rules.

In [23]:
print("cleaning data ...")
if len(trans_df) == 0 :
    trans_df = None
else :
    trans_df = trans_df[trans_df['translated_title'] != '']
    trans_df = trans_df[trans_df['translated_text'] != '']

    trans_df = trans_df[trans_df['translated_title'].str.count('\s+').ge(3)] #keep only titles having more than 4 spaces in the title
    trans_df = trans_df[trans_df['translated_text'].str.count('\s+').ge(20)] #keep only titles having more than 20 spaces in the body

    trans_df = (trans_df.drop_duplicates(subset=['translated_title'])).sort_index()
    trans_df = (trans_df.drop_duplicates(subset=['translated_text'])).sort_index()
    trans_df = (trans_df.drop_duplicates(subset=['links'])).sort_index()
    trans_df = trans_df.reset_index(drop=True)

    if len(trans_df) == 0 :
        trans_df = None

cleaning data ...


In [24]:
trans_df.shape

(194, 8)

# Embeddings 

In [25]:
texts = list(trans_df['translated_text'])

In [26]:
embedding = GoogleEmbeddings()
embedding.fit_transform(texts)

194
194


# Clustering 

In [27]:
## clustering train data 
xtrain = embedding.embedded_data
xtrain.shape

(194, 256)

In [28]:
model = Clustering(percentile=10, linkage='average', metric='cosine')
model.fit(xtrain, n=500)
ypred = model.predict(xtrain)

100%|██████████| 500/500 [00:05<00:00, 99.70it/s] 


In [29]:
trans_df['class'] = ypred
len(np.unique(ypred))

175

In [30]:
trans_df

,dates,titles,links,texts,lang,cat,translated_title,translated_text,class
0,2024-08-23T04:41:00Z,"Piloti Air Canada hrozí stávkou, chtějí stejné...",https://www.idnes.cz/ekonomika/doprava/air-can...,Piloti letecké společnosti Air Canada vstupují...,cs,stávka,"Air Canada pilots threaten to strike, want the...",Air Canada pilots go on strike alert. They are...,87
1,2024-08-23T01:13:17Z,Trabajadores del Poder Judicial protestan en C...,https://www.elfinanciero.com.mx/nacional/2024/...,Trabajadores del Poder Judicial se manifestaro...,es,protesta de empleados,Judicial Branch workers protest in the Chamber...,Workers of the Judicial Branch demonstrated ou...,144
2,2024-08-23T07:33:30Z,Le Seigneur des Anneaux : La Guerre du Rohirim...,https://www.journaldugeek.com/2024/08/23/le-se...,La Terre du Milieu sera à l’honneur en cette f...,fr,grève,The Lord of the Rings: War of the Rohirim Gets...,Middle-earth will be in the spotlight at the e...,168
3,2024-08-22T16:23:39Z,सुप्रीम कोर्ट की अपील के बाद काम पर लौटे डॉक्ट...,https://www.aajtak.in/india/news/story/doctors...,रेजिडेंट डॉक्टर्स एसोसिएशन एम्स (RDA AIIMS) दि...,hi,हड़ताल,Doctors returned to work after the Supreme Cou...,Resident Doctors Association AIIMS (RDA AIIMS)...,150
4,2024-08-23T03:08:21Z,न्याय की मांग को लेकर सीटू व जनवादी महिला समित...,https://www.dainiktribuneonline.com/news/harya...,"भिवानी, 22 अगस्त (हप्र) कामकाजी महिला समन्वय स...",hi,कर्मचारी विरोध प्रदर्शन,Demonstration by CITU and Janwadi Mahila Samit...,"Bhiwani, August 22 (HP) On the call of the Wor...",129
...,...,...,...,...,...,...,...,...,...
189,2024-08-23T10:30:01Z,HP Color Laser MFP 179fwg Multifunktions-Farbl...,https://www.amazon.de/dp/B07RMJV1LT?tag=winfud...,Freitag ab 12:30 Uhr: HP Color Laser MFP 179fw...,de,schlagen,HP Color Laser MFP 179fwg Multifunction color ...,Friday from 12:30 p.m.: HP Color Laser MFP 179...,17
190,2024-08-23T07:26:25Z,Bundesliga-Vorschau - Titelkandidaten im Meist...,https://www.focus.de/sport/fussball/bundesliga...,E-Mail\n\nTeilen\n\nMehr\n\nTwitter\n\nDrucken...,de,schlagen,Bundesliga preview - Title candidates in the c...,E-mail\n\nShare\n\nMore\n\nTwitter\n\nPrint\n\...,0
191,2024-08-23T11:42:00Z,Der besondere Ausblick auf die Liga - Kane-Kom...,https://www.focus.de/sport/fussball/bundesliga...,Der besondere Ausblick auf die Liga: Kane-Koma...,de,schlagen,The special outlook on the league - Kane coma ...,The special outlook on the league: Kane coma a...,18
192,2024-08-22T03:30:00Z,KOMMENTAR - Unsere Psyche wird immer kränker. ...,https://www.nzz.ch/meinung/unsere-psyche-wird-...,Kommentar Unsere Psyche wird immer kränker. Do...,de,schlagen,COMMENT - Our psyche is becoming increasingly ...,Comment Our psyche is becoming increasingly il...,12


# RAG

In [31]:
trans_df = trans_df.rename(columns= {'links': 'url', 'dates': 'date'})
rag_data = trans_df[['date', 'cat', 'lang', 'url','translated_title', 'translated_text', 'class']]
# rag_data = dataframe[['date', 'cat', 'lang', 'title', 'text', 'url','translated_title', 'translated_text', 'class', 'description']]

rag_data = rag_data.fillna('')

In [32]:
rag_data

,date,cat,lang,url,translated_title,translated_text,class
0,2024-08-23T04:41:00Z,stávka,cs,https://www.idnes.cz/ekonomika/doprava/air-can...,"Air Canada pilots threaten to strike, want the...",Air Canada pilots go on strike alert. They are...,87
1,2024-08-23T01:13:17Z,protesta de empleados,es,https://www.elfinanciero.com.mx/nacional/2024/...,Judicial Branch workers protest in the Chamber...,Workers of the Judicial Branch demonstrated ou...,144
2,2024-08-23T07:33:30Z,grève,fr,https://www.journaldugeek.com/2024/08/23/le-se...,The Lord of the Rings: War of the Rohirim Gets...,Middle-earth will be in the spotlight at the e...,168
3,2024-08-22T16:23:39Z,हड़ताल,hi,https://www.aajtak.in/india/news/story/doctors...,Doctors returned to work after the Supreme Cou...,Resident Doctors Association AIIMS (RDA AIIMS)...,150
4,2024-08-23T03:08:21Z,कर्मचारी विरोध प्रदर्शन,hi,https://www.dainiktribuneonline.com/news/harya...,Demonstration by CITU and Janwadi Mahila Samit...,"Bhiwani, August 22 (HP) On the call of the Wor...",129
...,...,...,...,...,...,...,...
189,2024-08-23T10:30:01Z,schlagen,de,https://www.amazon.de/dp/B07RMJV1LT?tag=winfud...,HP Color Laser MFP 179fwg Multifunction color ...,Friday from 12:30 p.m.: HP Color Laser MFP 179...,17
190,2024-08-23T07:26:25Z,schlagen,de,https://www.focus.de/sport/fussball/bundesliga...,Bundesliga preview - Title candidates in the c...,E-mail\n\nShare\n\nMore\n\nTwitter\n\nPrint\n\...,0
191,2024-08-23T11:42:00Z,schlagen,de,https://www.focus.de/sport/fussball/bundesliga...,The special outlook on the league - Kane coma ...,The special outlook on the league: Kane coma a...,18
192,2024-08-22T03:30:00Z,schlagen,de,https://www.nzz.ch/meinung/unsere-psyche-wird-...,COMMENT - Our psyche is becoming increasingly ...,Comment Our psyche is becoming increasingly il...,12


### Get suppliers data 


In [33]:
# load suppliers
supplier1 = pd.read_csv('/home/jupyter/news/data/suppliers/20231004_Fichier_Extraction_avec_usines_clientes.csv')
supplier1.drop(columns=["Unnamed: 0"], inplace=True)
supplier1['tier'] ='1'
# supplier1

supplier2 = pd.read_csv('/home/jupyter/news/data/suppliers/tiern_N.csv')
supplier2.drop(columns=["Unnamed: 0"], inplace=True)
supplier2['tier'] ='N'
# supplier2
suppliers = pd.concat([supplier1[['country', 'city', 'suggested_name', 'tier']], supplier2[['country', 'city', 'suggested_name', 'tier']]])
print(suppliers.shape)
suppliers.dropna(inplace=True)
print(suppliers.shape)

(59581, 4)
(59580, 4)


### Running RAG

In [34]:
strike = StrikeRAG(vertexai_llm= conf['rag_cong']['vertexai_llm'],
                   vertexai_embedding_name = conf['rag_cong']['vertexai_embedding_name'], 
                   retry = conf['rag_cong']['retry'], 
                   max_doc = conf['rag_cong']['max_doc'], 
                   chunk_size = conf['rag_cong']['chunk_size'], 
                   chunk_overlap = conf['rag_cong']['chunk_overlap']
                  )

strike.retrieve_infos_with_retry(dataframe=rag_data)

label : 0
 document creation : 0
first retrieval
company is None or belongs to self.liste
label : 1
 document creation : 1
first retrieval
company is None or belongs to self.liste
label : 2
 document creation : 2
first retrieval
second retrieval
3rd retrieval
results :
{'strike': {'labor_strike': 'no', 'justification': 'The text explicitly states that the shutdown is a lockout, where management prevents workers from working, rather than a strike where workers refuse to work.'}, 'impacted_company': 'Canadian National (CN)', 'locations': [{'city': 'Various', 'country': 'Canada'}], 'impacted_business_sectors': ['Transportation', 'Agriculture'], 'automotive_industry': {'concerned': 'yes', 'justification': "The article states that some U.S. auto plants could temporarily shut down if they can't manufacture engines, transmissions, or stampings in Canadian plants due to the disruption of supply chains caused by the rail shutdown. This indicates that the car-making industry is directly affected

In [35]:
iterative_results = deepcopy(strike.all_results)
len(iterative_results)

28

### Matching process 


In [36]:
matching =  Matching()
r = matching.match(set_news = iterative_results, dataframe=suppliers)

28it [00:00, 39.90it/s]


### Filtering process

In [37]:
# SECTORS_TO_DISCARD = sectors_to_discard.SECTORS_TO_DISCARD

In [38]:
# decision_function_args = {
#     'sectors_to_discard': dict_config["sectors_to_discard"],
#     'desirable_temporalities' : ['upcoming', 'ongoing', 'unknown']
# }

In [39]:
filtre = Filter(empty_companie_name_index=matching.index, decision_function=conf['decision_function'], decision_function_args=conf["decision_function_args"], filename=None)
rr = filtre.filtering(matching.results)

In [40]:
rr

[{'strike': {'labor_strike': 'yes',
   'justification': 'The provided context explicitly states that the strike at Tesla in Sweden was initiated by IF Metall, a labor union, and involved other unions in sympathy measures. This indicates a labor strike driven by union demands for a collective agreement.'},
  'impacted_company': 'Tesla',
  'locations': [{'city': 'Norrköping', 'country': 'Sweden'}],
  'impacted_business_sectors': ['Automotive Manufacturing', 'Technology'],
  'automotive_industry': {'concerned': 'yes',
   'justification': "Tesla is a major player in the automotive manufacturing industry. A strike at Tesla, especially one that disrupts production, would likely have a ripple effect on the broader car-making industry. This is because:\n\n* **Supply Chain Disruptions:** Tesla's strike could disrupt the supply of components or materials used by other car manufacturers, especially those relying on similar parts or technologies.\n* **Competition:**  A strike at Tesla could give c

## Grouping processed articles based on their names

In [41]:
rr = groupByName(rr)

In [42]:
results = generate_description(rr, dataframe=rag_data)

yes


1it [00:00,  1.30it/s]


## Adding geographical coordonate

In [43]:
load_dotenv()
NEWS_API_KEY = os.getenv('NEWS_API_KEY')
os.environ['http_proxy'] = os.getenv('http_proxy')
os.environ['https_proxy'] = os.getenv('https_proxy')
os.environ['HTTP_PROXY'] = os.getenv('HTTP_PROXY')
os.environ['HTTPS_PROXY'] = os.getenv('HTTPS_PROXY')

In [44]:
final_results = geoloc(results)

In [45]:
final_results

[{'strike': {'labor_strike': 'yes',
   'justification': 'The provided context explicitly states that the strike at Tesla in Sweden was initiated by IF Metall, a labor union, and involved other unions in sympathy measures. This indicates a labor strike driven by union demands for a collective agreement.'},
  'impacted_company': 'Tesla',
  'locations': [{'city': 'Norrköping',
    'country': 'Sweden',
    'latitude': 58.5909124,
    'longitude': 16.1903511}],
  'impacted_business_sectors': ['Automotive Manufacturing', 'Technology'],
  'automotive_industry': {'concerned': 'yes',
   'justification': "Tesla is a major player in the automotive manufacturing industry. A strike at Tesla, especially one that disrupts production, would likely have a ripple effect on the broader car-making industry. This is because:\n\n* **Supply Chain Disruptions:** Tesla's strike could disrupt the supply of components or materials used by other car manufacturers, especially those relying on similar parts or tech

## Saving the final results

In [46]:
# file path where the results will be saved
filename = "/home/jupyter/news/results/json/strike/complete_pipeline_test.json"
with open(filename, "w") as final:
    json.dump(final_results, final, indent=4)
    print("saved !")

saved !
